In [2]:
# ---- DEMO File ----
# ES170 Final Project

# Eric Puma  
# James Baskerville 
# Peter Seung Min Lee 
# Francisco Trujillo

# Prof. Prineha Narang
# Harvard University 2018

import numpy as np
from qutip import *
from scipy.optimize import minimize
import h5py
import matplotlib.pyplot as plt
import sys
import pickle
import time

'''Returns the identity operator in a Hilbert Space off size n'''
def tenseye(n):
    return qeye([2] * n)

'''Returns the operator composed by a set of gates in a HS of size n'''
def gate_prod(n, gates):
    prod = tenseye(n)
    for gate in gates:
        prod = prod * gate
    return prod

'''Returns an operator corresponding tothe ZYZ rotations defined by 
angles in <params> applied only to the target qubit <tgt> in an 
Hilbert Space of dimension <n>'''
def rot(n, params, tgt):
    alpha, theta, beta = params
    return rz(alpha, n, tgt) * ry(theta, n, tgt) * rz(beta, n, tgt)

'''Returns an operator corresponding to a set of rotations conditional 
on the qubit <ctrl> defined by angles in <params> applied only to 
the target qubit <tgt> in an Hilbert Space of dimension <n>.'''
def ctrl_rot(n, params, ctrl, tgt):
    alpha, theta, beta = params
    A = rz(alpha, n, tgt) * ry(theta / 2.0, n, tgt)
    B = ry(-theta / 2.0, n, tgt) * rz(-(alpha + beta) / 2.0, n, tgt)
    C = rz((beta - alpha) / 2.0, n, tgt)    
    return A * cnot(n, ctrl, tgt) * B * cnot(n, ctrl, tgt) * C

'''Returns the number of gates and parameters needed for <n_qubits>.'''
def init_consts(n_qubits):
    n_gates = n_qubits ** 2 + n_qubits
    n_params = n_gates * 3
    return n_gates, n_params

'''Returns an array of size <n_params> initialized with <method>'''
def init_params(n_params, method=np.zeros):
    return method(n_params)

'''Returns three separate arrays representing the parameters 
for the first set of unitary rotations, the controlled rotations, 
and the last set of unitary rotations defined in <params> where
<n_qs> is the size of the circuit's Hilbert space.'''
def split_params(n_qs, params):
    return (params[:3*n_qs].reshape(n_qs, 3),
            params[3*n_qs:-3*n_qs].reshape(n_qs, n_qs-1, 3),
            params[-3*n_qs:].reshape(n_qs, 3))

'''Flatten and com'''
def recombine_params(first, mid, last):
    return np.concatenate((first.flatten(), mid.flatten(), last.flatten()))
